Where the actual experiment happens!

In [3]:
#import the repository
!git clone https://github.com/steinburglar/REAN
%cd REAN


Cloning into 'REAN'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 163 (delta 31), reused 148 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (163/163), 22.77 MiB | 19.83 MiB/s, done.
Resolving deltas: 100% (31/31), done.
[Errno 2] No such file or directory: 'yourrepo'
/content


In [8]:
%cd /content/REAN

/content/REAN


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:

#do necessary imports
from pathlib import Path
import sys
import torch
import json
import torch.nn as nn
import torch.optim as optim
from rean.utils import make_run_dir, to_serializable
from rean.data.Dataset import make_datasets
from rean.models.CNN import PlainCNN
from rean.models.P4 import P4CNN
from rean.models.RelaxedP4 import RelaxedP4CNN
from rean.train import train_full, evaluate
from rean.plot import LossPlot
import matplotlib.pyplot as plt


In [22]:
#some params that should be the same across all experimental runs

group_order = 4
hidden_dim = 20
out_channels = hidden_dim
classes = 10
kernel_size = 3
num_gconvs = 6
num_epochs = 10
batch_size = 64
learning_rate = 0.002
gamma = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [23]:
def make_run_dir(model_name,
                 noise_type,
                 noise_params,
                 learning_rate,

                 ):
    gamma = noise_params.get("gamma")
    std = noise_params.get("std")
    run_dir = Path(f"/content/drive/MyDrive/runs/{model_name}_{noise_type}_std{std}_gamma{gamma}_lr{learning_rate}")
    run_dir.mkdir(parents=True, exist_ok=True)
    return run_dir

# Experiment Running Loop #

In [21]:
#test full loop
models = ['P4CNN', 'RelaxedP4CNN']
noise_types = ['none', 'iso', 'aniso']
stds = [0.1, 0.2, 0.3]

for model_name in models:
    for noise_type in noise_types:
        for i, std in enumerate(stds):
            if noise_type == 'none':
                if i != 0: #only run noiseless once
                    continue
                train_noise = None
                noise_params = {"mean":0, "std":0, 'gamma':0}

            else:
                train_noise = noise_type
                test_noise = noise_type #could be none?
                noise_params = {'mean': 0, 'std': std, 'gamma': gamma}
            #make datasets
            train_ds, val_ds, test_ds_noised, in_channels = make_datasets(train_noise=train_noise,
                                                                   test_noise = test_noise,
                                                            noise_params=noise_params,
                                                            group_order=group_order)
            _, _, test_ds_clean, _ = make_datasets(test_noise=None, noise_params=noise_params,
                                                            group_order=group_order)
            #train model
            run_data, best_model = train_full(model_name=model_name,
                          train_ds=train_ds,
                          val_ds=val_ds,
                          in_channels=in_channels,
                          hidden_dim=hidden_dim,
                          out_channels=out_channels,
                          classes=classes,
                          num_gconvs=num_gconvs,
                          kernel_size=kernel_size,
                          group_order=group_order,
                          num_epochs=num_epochs,
                          batch_size=batch_size,
                          learning_rate=learning_rate,
                          device=device)
            run_data['noise_type'] = noise_type
            run_data['std'] = std

            run_dir = make_run_dir(model_name, noise_type, noise_params = noise_params, learning_rate = learning_rate)
            torch.save(best_model.state_dict(), run_dir)
            with open(run_dir / "run_data.json", "w") as f:
                json.dump(run_data, f, default = to_serializable, indent=2)

            #test the model
            test_noisy_loader = torch.utils.data.DataLoader(test_ds_noised, batch_size=batch_size, shuffle=False, pin_memory =True)
            test_clean_loader = torch.utils.data.DataLoader(test_ds_clean, batch_size=batch_size, shuffle=False, pin_memory =True)
            criterion = nn.CrossEntropyLoss()
            test_loss, test_acc = evaluate(best_model,  device, test_noisy_loader, criterion = criterion)
            test_loss_clean, test_acc_clean = evaluate(best_model,  device, test_clean_loader, criterion = criterion)
            test = {
                "test_noisy_loss": test_loss,
                "test__noisy_acc": test_acc,
                "test_clean_loss": test_loss_clean,
                "test_clean_acc": test_acc_clean
            }
            print(f"Test Loss: {test_loss}, Test Acc: {test_acc}")
            with open(run_dir / "test_data.json", "w") as f:
                json.dump(test, f, default = to_serializable, indent=2)



25400 trainable parameters in P4CNN model
Epoch 1/40, Train Loss: 1.3796, Val Loss: 0.7328, Val Acc: 74.32%
Epoch 2/40, Train Loss: 0.5954, Val Loss: 0.4277, Val Acc: 86.90%
Epoch 3/40, Train Loss: 0.3735, Val Loss: 0.4723, Val Acc: 87.27%
Epoch 4/40, Train Loss: 0.2939, Val Loss: 0.2620, Val Acc: 91.80%
Epoch 5/40, Train Loss: 0.2414, Val Loss: 0.2621, Val Acc: 92.14%
Epoch 6/40, Train Loss: 0.2127, Val Loss: 0.1836, Val Acc: 94.49%
Epoch 7/40, Train Loss: 0.2035, Val Loss: 0.1841, Val Acc: 94.41%
Epoch 8/40, Train Loss: 0.1869, Val Loss: 0.3200, Val Acc: 90.21%


KeyboardInterrupt: 

# Plotting #


In [ ]:
#first, plot the performance of all 3 models on the baseline test set, on two sets of axes: one for train, one for validation
fig, (train_ax, val_ax) = plt.subplots(1,2)
runs = []
tests = []
for model_name in models:
    runpath = Path(f"./runs/{model_name}_none_std0_gamma0_lr{learning_rate}")

    #load the rundata in from the json
    rundatapath = runpath / "run_data.json"
    testdatapath = runpath / "test_data.json"
    with rundatapath.open("r", encoding="utf-8") as f:
        run_data = json.load(f)
    with testdatapath.open("r", encoding="utf-8") as f:
        test_data = json.load(f)
    run_data.update(test_data)
    runs.append(run_data)
    tests.append(test_data)

#make loss plots
train_ax = LossPlot(train_ax, runs, labels = ["model_name"], title = "Training Loss", val = False)
val_ax = LossPlot(val_ax, runs, labels = ['model_name'], title = "Validation Loss", train = False)
fig.tight_layout()
fig.suptitle("Training and Validation Loss, Baseline (No Noise) Experiments", y=1.02)
fig.savefig("baseline_loss_plots.png")







In [ ]:
# do the same for each noise type, but plotting both std runs on the same axes
#so, each axes object will have 6 lines: 3 models x 2 stds
noise_types = ['iso', 'aniso']
for noise_type in noise_types:
    fig, (train_ax, val_ax) = plt.subplots(1,2)
    runs = []
    tests = []
    for model_name in models:
        for std in stds:
            runpath = Path(f"./runs/{model_name}_{noise_type}_std{std}_gamma{gamma}_lr{learning_rate}")

            #load the rundata in from the json
            rundatapath = runpath / "run_data.json"
            testdatapath = runpath / "test_data.json"
            with rundatapath.open("r", encoding="utf-8") as f:
                run_data = json.load(f)
            with testdatapath.open("r", encoding="utf-8") as f:
                test_data = json.load(f)
            run_data.update(test_data)
            runs.append(run_data)
            tests.append(test_data)

    #make loss plots
    train_ax = LossPlot(train_ax, runs, labels = ["model_name", "std"], title = "Training Loss", val = False)
    val_ax = LossPlot(val_ax, runs, labels = ['model_name', "std"], title = "Validation Loss", train = False)
    fig.tight_layout()
    fig.suptitle(f"Training and Validation Loss, {noise_type.capitalize()} Noise Experiments", y=1.02)
    fig.savefig(f"{noise_type}_loss_plots.png")


In [ ]:
#fianlly, generate a dataframe for test accuracies across all runs
import pandas as pd
all_runs = []
for model_name in models:
    for noise_type in noise_types + ['none']:
        if noise_type == 'none':
            runpath = Path(f"./runs/{model_name}_none_std0_gamma0_lr{learning_rate}")
            std = 0
        for std in stds:
            runpath = Path(f"./runs/{model_name}_{noise_type}_std{std}_gamma{gamma}_lr{learning_rate}")
            #load the rundata in from the json
            rundatapath = runpath / "run_data.json"
            testdatapath = runpath / "test_data.json"
            with rundatapath.open("r", encoding="utf-8") as f:
                run_data = json.load(f)
            with testdatapath.open("r", encoding="utf-8") as f:
                test_data = json.load(f)
            run_data.update(test_data)
            all_runs.append(run_data)
df = pd.DataFrame(all_runs)
df = df[['model_name', 'noise_type', 'std', 'test_acc']] #get only the stuff we want
df.to_csv("test_accuracies.csv", index=False)